In [1]:
%%HTML
<style>.container{width:100%}</style>

In [2]:
def id2bit(ls:list):
    if(len(ls)==0):
        return [0,0,0,0, 0,0,0,0]  # Return a default 8-bit array
    aa = [0 for i in range(max(ls)+1)]
    for i in ls:
        aa[i] = 1
    return aa[::-1]

def bit2id(ls:list):
    ls = ls[::-1]
    aa = []
    for i in range(len(ls)):
        if(ls[i] == 1):
            aa.append(i)
    return aa[::-1]
    
def XOR(*args):
    result = 0
    for arg in args:
        result ^= arg
    return result
    
class LFSR:
    def __init__(self,start,poly):
        self.seq = start
        self.taps = bit2id(poly[:-1])  # ignore the output tap (final bit)

        if len(self.seq) != len(poly) - 1:
            raise ValueError("Polynomial and start value length mismatch")

    def clock(self):
        # print(self.seq) 
        feedback = XOR(*[self.seq[bit] for bit in self.taps])
        
        self.seq = [feedback] + self.seq[:-1]

        
class A51:
    def __init__(self,lfsrs,clock_bits):
        self.lfsrs = lfsrs
        self.clock_bits = clock_bits
        self.lfsr_count = len(clock_bits)
    def majority(self, *bits):
        ones = sum(i for i in bits if i==1)
        if ones >= self.lfsr_count / 2:
            majority_bit = 1
        else:
            majority_bit = 0
        return majority_bit
        
    def clock(self):
        majority = self.majority(*[self.lfsrs[i].seq[self.clock_bits[i]] for i in range(self.lfsr_count)])
        for i in range(self.lfsr_count):
            if(self.lfsrs[i].seq[self.clock_bits[i]] == majority):
                self.lfsrs[i].clock()
        out = XOR(*[int(i.seq[-1]) for i in self.lfsrs])
        return out
        
lf1 = LFSR(start=[1,0,1,1],poly=id2bit([4,1]))
lf2 = LFSR(start=[0,1,1,1],poly=id2bit([4,1]))
lf3 = LFSR(start=[1,0,1,0],poly=id2bit([4,1]))
a51 = A51(lfsrs=[lf1,lf2,lf3],clock_bits=[1,2,0])
stream = [a51.clock() for i in range(10)]
stream

[1, 0, 0, 0, 0, 1, 0, 1, 1, 0]

In [3]:
import os
def write2txt_file(bitstream, filename):
    """Writes a bitstream (string of '0's and '1's) to a text file."""
    with open(filename, 'a') as f:  # Open in append mode to continue writing
        f.write(bitstream)

def write2bin_file(bitstream, filename):
    byte_list = []
    
    # Pad the bitstream if it's not a multiple of 8
    padding = (8 - (len(bitstream) % 8)) % 8
    bitstream += '0' * padding  # Add extra '0's to make the length a multiple of 8
    
    for i in range(0, len(bitstream), 8):
        byte = bitstream[i:i+8]
        byte_list.append(int(byte, 2))  # Convert 8 bits to an integer (byte)
    
    # Append the bytes to the binary file
    with open(filename, 'ab') as f:  # 'ab' mode to append to the binary file
        f.write(bytearray(byte_list))

def gen_bit_stream(data:dict,target_size,file_path):
    lfsrs = [LFSR(start=i["start"],poly=i["poly"]) for i in data]
    a51 = A51(lfsrs=lfsrs,clock_bits=[i["clock"] for i in data])
    
    # filename = 'bitstream_output_1GB.bin'
    # target_size = 1 * 1024 * 1024 * 1024  # 1 GB in bytes
    current_size = 0

    bitstream_chunk = ""  # Chunk of bits to write periodically
    chunk_size = 10000  # Number of bits to generate at a time (can adjust for performance)
    progress_interval = target_size // 10  # 1/10th of the target size (100 MB)
    next_progress_checkpoint = progress_interval
    
    while current_size < target_size:
        # Generate bits in chunks
        for _ in range(chunk_size):
            bitstream_chunk += str(a51.clock())

        # Write chunk to file
        # write2bin_file(bitstream_chunk, filename)
        write2txt_file(bitstream_chunk, file_path)
        
        # Clear the chunk and update the current file size
        bitstream_chunk = ""
        current_size = os.path.getsize(file_path)
        # Check if the file size has crossed the 1/10th checkpoint
        if current_size >= next_progress_checkpoint:
            print(f"File size crossed {round(next_progress_checkpoint / (1024 * 1024),2)} MB")
            next_progress_checkpoint += progress_interval  # Update to next 10% checkpoint

       

    print(f"File generation complete: {file_path} (target)")

In [4]:
data = [
    {"start": [0, 1, 0, 1, 1], "poly": id2bit([5, 2, 0]), "clock": 2},
    {"start": [1, 0, 0, 1, 0], "poly": id2bit([5, 4, 3, 1, 0]), "clock": 3},
    {"start": [0, 1, 1, 0, 0], "poly": id2bit([5, 4, 2, 1, 0]), "clock": 2}
]
gen_bit_stream(data, target_size=1*1024**2, file_path="mine_gen_100MB.txt")

File size crossed 0.1 MB
File size crossed 0.2 MB
File size crossed 0.3 MB
File size crossed 0.4 MB
File size crossed 0.5 MB
File size crossed 0.6 MB
File size crossed 0.7 MB
File size crossed 0.8 MB
File size crossed 0.9 MB
File size crossed 1.0 MB
File generation complete: mine_gen_100MB.txt (target)
